# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd108f55220>
├── 'a' --> tensor([[-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445]])
└── 'x' --> <FastTreeValue 0x7fd108f55850>
    └── 'c' --> tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                        [-0.0141, -0.9378,  0.2930, -0.8671],
                        [ 0.4136,  0.3118, -1.8769, -1.0229]])

In [4]:
t.a

tensor([[-1.5307, -0.5999,  0.1922],
        [ 1.3266, -0.3408,  0.1445]])

In [5]:
%timeit t.a

62.6 ns ± 0.102 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd108f55220>
├── 'a' --> tensor([[ 3.1249, -1.4667,  0.1918],
│                   [ 0.4995, -0.2743,  1.3068]])
└── 'x' --> <FastTreeValue 0x7fd108f55850>
    └── 'c' --> tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                        [-0.0141, -0.9378,  0.2930, -0.8671],
                        [ 0.4136,  0.3118, -1.8769, -1.0229]])

In [7]:
%timeit t.a = new_value

70.4 ns ± 0.0605 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.5307, -0.5999,  0.1922],
               [ 1.3266, -0.3408,  0.1445]]),
    x: Batch(
           c: tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229]]),
       ),
)

In [10]:
b.a

tensor([[-1.5307, -0.5999,  0.1922],
        [ 1.3266, -0.3408,  0.1445]])

In [11]:
%timeit b.a

58.7 ns ± 0.0583 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1054,  1.2166, -0.9555],
               [ 0.5881,  0.6057, -0.6482]]),
    x: Batch(
           c: tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229]]),
       ),
)

In [13]:
%timeit b.a = new_value

496 ns ± 1.57 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

879 ns ± 0.258 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.8 µs ± 25.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

144 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 372 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd05bc07a90>
├── 'a' --> tensor([[[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]],
│           
│                   [[-1.5307, -0.5999,  0.1922],
│                    [ 1.3266, -0.3408,  0.1445]]])
└── 'x' --> <FastTreeValue 0x7fd0d7f5d3d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.6 µs ± 59.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd05bc07df0>
├── 'a' --> tensor([[-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445],
│                   [-1.5307, -0.5999,  0.1922],
│                   [ 1.3266, -0.3408,  0.1445]])
└── 'x' --> <FastTreeValue 0x7fd05bc07550>
    └── 'c' --> tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                        [-0.0141, -0.9378,  0.2930, -0.8671],
                 

In [23]:
%timeit t_cat(trees)

29.7 µs ± 56.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.4 µs ± 68.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2967, -0.5528,  0.9899, -1.4097],
                       [-0.0141, -0.9378,  0.2930, -0.8671],
                       [ 0.4136,  0.3118, -1.8769, -1.0229]],
              
                      [[-0.2967, -0.5528,  0.9899, -1.4097],
                       [-0.0141, -0.9378,  0.2930, -0.8671],
                       [ 0.4136,  0.3118, -1.8769, -1.0229]],
              
                      [[-0.2967, -0.5528,  0.9899, -1.4097],
                       [-0.0141, -0.9378,  0.2930, -0.8671],
                       [ 0.4136,  0.3118, -1.8769, -1.0229]],
              
                      [[-0.2967, -0.5528,  0.9899, -1.4097],
                       [-0.0141, -0.9378,  0.2930, -0.8671],
                       [ 0.4136,  0.3118, -1.8769, -1.0229]],
              
                      [[-0.2967, -0.5528,  0.9899, -1.4097],
                       [-0.0141, -0.9378,  0.2930, -0.8671],
                       [ 0.4136,  0.3118, -1.8769, -1.0229]],

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 764 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229],
                      [-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229],
                      [-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229],
                      [-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229],
                      [-0.2967, -0.5528,  0.9899, -1.4097],
                      [-0.0141, -0.9378,  0.2930, -0.8671],
                      [ 0.4136,  0.3118, -1.8769, -1.0229],
                      [-0.2967, -0.5528,  0.9899, -1.4097],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

332 µs ± 3.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
